In [13]:
import spacy
nlp = spacy.load('en')

In [112]:
s1="In April 1990, Universal Studios bought the rights to develop Iron Man for the big screen, with Stuart Gordon to direct a low-budget film based on the property."
s2="In February 1996, Fox Media had acquired the rights from Universal for $3000."
s3="In August 2015, Fox Networks acquired the US cable broadcast rights, for broadcast after its theatrical release."
s4="In early 2003, actor-director Clint Eastwood and production people at the studio Warner Bros. bought the film rights to James R. Hansen's First Man: The Life of Neil A. Armstrong."
s5="In 2006 Coppola contacted Steven Spielberg—whose studio DreamWorks had recently been bought out by Paramount—about restoring The Godfather."
# bought out forms different pattern
s6="Shortly after, Sony Pictures Worldwide Acquisitions acquired distribution rights to the film for $5 million."
s7="Liberty Films was purchased by Paramount Pictures, and remained a subsidiary until 1951."
s8="The story came to the attention of RKO producer David Hempstead, who showed it to Cary Grant's Hollywood agent, and in April 1944, RKO Pictures bought the rights to the story for $10,000, hoping to turn the story into a vehicle for Grant."
#no seller detected. Because sementic meaning parsing for complex such complex statement is not done.
s9="M. & A. Alexander purchased the movie."
s10="Michael travels to Las Vegas to buy out Greene's stake in the family's casinos."

d1 = "The rights of Iron Man were sold to Universal Studios for by Fox Media for $3,000 in May, 2015 at Prenston Theater."


In [117]:
match_buy_template(s1)

Buyer:Universal Studios
Product:the rights
Seller:None
Amount:<no-match>
Time:April 1990
Location:<no-match>


In [96]:
def match_buy_template(sent):
    result = pattern1Match(sent)
    
    if result == None:
        result = pattern2Match(sent)
        
    printResult(result)

In [115]:
def pattern1Match(sent):
    """
    this should match pattern "As of December 6, 2018, A Star Is Born has grossed $194.4 million in the United States and Canada, and $170.1 million in other territories, for a total worldwide gross of $364.5 million, against a production budget of $36–40 million."
    it need profit/earn/make/gross as verb 
    in addition money should exist in statement
    movie/scene can be attached as subject of verb
    """
    buyToken = getBuyVerbToken(sent)
    if(buyToken == None):
#         print("#1")
        return
    else:
        moneyToken = findChildMoneyToken(sent, buyToken)
        amount = "<no-match>"
        if moneyToken != None:
#             print("#2")
            amount = getMoneyThatContainsToken(sent, moneyToken)
        else:
            amount = getFirstEntity(sent, ["MONEY"])
            
        buyer = getAgentOfVerbToken(sent, buyToken)
#         print("Buyer:"+str(buyer))
        location = "<no-match>"
        if moneyToken != None:
#             print("#3")
            location = getLocationOfToken(sent, moneyToken)
        if location == "<no-match>":
#             print("#4")
            location = getLocationOfToken(sent, buyToken)
#         print("Location:"+str(location))
        
        product = getThemeOfVerbToken(sent, buyToken)
#         print("Product:"+str(product))
        
        seller = getPhraseOfFromChild(sent)
#         print("Seller:"+str(seller))
        
        time = getTimeOfToken(sent, buyToken)
#         print("time so far:"+str(time))
        if time == "<no-match>":
#             print("Looking for any TIME entity :"+str(time))
            time = getFirstEntity(sent, ["TIME","DATE"])
#         print("Time"+str(time))
        return [buyer, product, seller, amount, time, location]

In [72]:
match_buy_template(s4)

Buyer:actor-director Clint Eastwood
Product:the film rights
Seller:None
Amount:<no-match>
Time:early 2003
Location:<no-match>


In [102]:
def pattern2Match(sent):
    """
    this should match pattern "Rights were sold to Fox Media"
    it need profit/earn/make/gross as verb 
    in addition money should exist in statement
    movie/scene can be attached as subject of verb
    """
    sellToken = getSellVerbToken(sent)
    if(sellToken == None):
#         print("#1")
        return
    else:
        moneyToken = findChildMoneyToken(sent, sellToken)
        amount = "<no-match>"
        if moneyToken != None:
#             print("#2")
            amount = getMoneyThatContainsToken(sent, moneyToken)
        else:
            amount = getFirstEntity(sent, ["MONEY"])
        
        seller = getAgentOfVerbToken(sent, sellToken)

        location = "<no-match>"
        if moneyToken != None:
#             print("#3")
            location = getLocationOfToken(sent, moneyToken)
        if location == "<no-match>":
#             print("#4")
            location = getLocationOfToken(sent, sellToken)
#         print("Location:"+str(location))
        
        product = getThemeOfVerbToken(sent, sellToken)
#         print("Product:"+str(product))
        
        buyer = getPhraseOfToChild(sent)
#         print("Seller:"+str(seller))
        
        time = getTimeOfToken(sent, sellToken)
#         print("time so far:"+str(time))
        if time == "<no-match>":
#             print("Looking for any TIME entity :"+str(time))
            time = getFirstEntity(sent, ["TIME","DATE"])
#         print("Time"+str(time))
        return [buyer, product, seller, amount, time, location]

In [106]:
def getAgentOfVerbToken(sent, verbToken):
    for child in verbToken.children:
        if(child.dep_ == "agent"): #for "by"
            for subChild in child.children:
                return getPhraseThatContainsToken(sent, subChild)

    for child in verbToken.children:
        if(child.dep_ == "pobj" or child.dep_ == "nsubj"):
            return getPhraseThatContainsToken(sent, child)

In [41]:
def getThemeOfVerbToken(sent, verbToken):
    for child in verbToken.children:
        if(child.dep_ == "dobj" or child.dep_ == "nsubjpass"):
            return getPhraseThatContainsToken(sent, child)

In [49]:
def getBuyVerbToken(sent):
    """
    This will look for frequent verb of "buy", "acquire", "purchase"
    Other synonyms can be used from wrodnet but we extracted a subset of synonys based on domain.
    """
    for token in nlp(sent):
        if "VERB" == token.pos_ and token.lemma_ in ["buy", "acquire", "purchase"]:
            return token

In [93]:
def getSellVerbToken(sent):
    """
    This will look for frequent verb of "sell"
    Other synonyms can be used from wrodnet but we extracted a subset of synonys based on domain.
    """
    for token in nlp(sent):
        if "VERB" == token.pos_ and token.lemma_ in ["sell"]:
            return token

In [51]:
def getPhraseOfFromChild(sent):
    if " from " in sent:
        for child in nlp(sent):
            if child.text == "from":
                for grandChild in child.children:
                    return getPhraseThatContainsToken(sent, grandChild)

In [104]:
def getPhraseOfToChild(sent):
    if " to " in sent:
        for child in nlp(sent):
            if child.text == "to":
                for grandChild in child.children:
                    return getPhraseThatContainsToken(sent, grandChild)

In [71]:
def printResult(result):
    #[buyer, product, seller, amount, time, location]
    if(result != None):
        print("Buyer:"+str(result[0]))
        print("Product:"+str(result[1]))
        print("Seller:"+str(result[2]))
        print("Amount:"+str(result[3]))
        print("Time:"+str(result[4]))
        print("Location:"+str(result[5]))
    else:
        print("No Parse")

In [21]:
def getTimeOfToken(sent, token):
    for child in token.children: 
                #usually time starts with "on", "in", "at"
                if child.text in ["on","at","in"]:
                    #usually "on"'s child is [August] 
#                     print("checking children of "+child.text)
                    for subChild in child.children:
                        temporalEntity = getTemporalThatContainsToken(sent, subChild)
                        if(temporalEntity != None):
                            return temporalEntity
    return "<no-match>"

In [22]:
def getTemporalThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and ent.label_ == "DATE" and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return None

In [23]:
import re
def calculateStartPositionOfToken(statement, targetToken):
    """
    this should return start position of the token
    """
    matches = [m.start() for m in re.finditer(targetToken.text, statement)]
    if(len(matches)==1):
        return matches[0]
    

    huristicPosition = 0
    doc = nlp(statement)
    
    i = 0
    for token in doc:
        if i < targetToken.i:
            huristicPosition = huristicPosition + len(token.text)+1
            i = i+1
        else:
            break
        
    distances = []
    for match in matches:
        distances.append(abs(match-huristicPosition))
        
    minIndex = distances.index(min(distances))
    
    return matches[minIndex]

In [24]:
def getLocationOfToken(sent, token):
    for child in token.children: 
        #usually release location starts with "in"
        if child.text in ["in","throughout", "at"]:
            for subChild in child.children:
#                 print("in loop for:"+subChild.text)
                GPEEntity = getLocationThatContainsToken(sent, subChild)
                if GPEEntity == None:
#                     print("no gpe found")
                    #this might be slipplary when "at" comes. Might confuse with time.
                    canConfuseWithTime = (getTemporalThatContainsToken(sent, subChild) != None)
#                     print("Can confuse:"+subChild.text+":"+str(canConfuseWithTime))
                    if not canConfuseWithTime:
                        GPEEntity = getPhraseThatContainsToken(sent, subChild)
                    
                if GPEEntity != None:
                    return GPEEntity
    return "<no-match>"

In [25]:
def getLocationThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and ent.label_ != "TIME" and ent.label_ != "DATE"  and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return None

In [26]:
def getPhraseThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for chunk in doc.noun_chunks:
        if(token.text in chunk.text and tokenStartPos >= chunk.start_char and tokenStartPos <= chunk.end_char):
            return chunk.text
    return None

In [27]:
def getEntityRelatedByOF(sent, token):
    for child in token.children: 
                #usually movie name for noun attached with "on"
                if child.text in ["of"]:
                    #['of', 'prep', 'premiere', 'NOUN', [IronMan]]
                    for subChild in child.children:
                        entity = getEntityThatContainsToken(sent, subChild)
                        if entity == None:
                            entity = getPhraseThatContainsToken(sent, subChild)
                        if(entity != None):
                            return entity
    return "<no-match>"

In [28]:
def getEntityThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return None

In [29]:
def getFirstGPE(sent):
    doc = nlp (sent)
    for ent in doc.ents:
        if(ent.label_ == "GPE"):
            return ent.text
    return "<no-match>"

In [60]:
def getFirstEntity(sent, expectedLabels):
    doc = nlp(sent)
    for ent in doc.ents:
        if(ent.label_ in expectedLabels):
            return ent.text
    return "<no-match>"

In [31]:
def findChildMoneyToken(sent, token):
    for child in token.children:
        if getMoneyThatContainsToken(sent, child) != None:
            return child
    return None

In [32]:
def getMoneyThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and ent.label_ == "MONEY" and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return None

In [33]:
def doesTokenChildrenIncludeWorldWide(token):
    for child in token.children:
        if "worldwide" in child.text.lower():
            return True
    return False

In [34]:
def getSubjectTokenOfVerb(verbToken):
    for child in verbToken.children:
        if "subj" in child.dep_:
            return child
    return None

In [36]:
def getEntityRelatedByBy(sent, token):
    for child in token.children: 
                if child.text in ["by"]:
                    for subChild in child.children:
                        entity = getEntityThatContainsToken(sent, subChild)
                        if(entity != None):
                            return entity
    return "<no-match>"